In [104]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics, svm

In [86]:
path = 'DataSet_w_NA (1).xlsx'
df = pd.read_excel(path, sheet_name='Испорченные факты')

In [87]:
df.head()

,Номер заказа,Возврат,Дата заказа,Дата отправки,Режим отправки,Факты.Контрагент ID,Факты.Товар ID,"Продажи, руб","Продажи, шт",Скидка,Маржинальная прибыль,Повторение заказа,Повторение контрагента,Повторение товара
0,2014-100279,Нет,2015-09-10 00:00:00,2015-09-14 00:00:00,Стандарт,CLNT0000422,PROD0000870,11190.0,2.0,0.0,5371.20,1,7,3
1,2014-100293,Нет,2015-09-14 00:00:00,2015-09-18 00:00:00,Стандарт,CLNT0000784,PROD0000646,45528.0,6.0,0.2,15934.80,1,4,8
2,2014-100328,Нет,2015-07-28 00:00:00,2015-08-03 00:00:00,Стандарт,CLNT0000342,PROD0000187,1964.0,1.0,0.2,662.85,1,4,12
3,2014-100363,Нет,2015-10-08 00:00:00,2015-10-15 00:00:00,Стандарт,CLNT0000146,PROD0001095,1184.0,2.0,0.2,414.40,2,12,7
4,2014-100363,Нет,2015-10-08 00:00:00,2015-10-15 00:00:00,Стандарт,CLNT0000146,PROD0001616,9504.0,3.0,0.2,3445.20,2,12,3


In [88]:
df.shape

(10293, 14)

In [89]:
df.isnull().sum()

Номер заказа                0
Возврат                     0
Дата заказа                 0
Дата отправки               0
Режим отправки              0
Факты.Контрагент ID        90
Факты.Товар ID             51
Продажи, руб              195
Продажи, шт                21
Скидка                      0
Маржинальная прибыль       18
Повторение заказа           0
Повторение контрагента      0
Повторение товара           0
dtype: int64

In [91]:
df = df.dropna()

In [92]:
df = pd.pivot_table(df, values=['Продажи, руб', 'Продажи, шт','Повторение заказа', 'Маржинальная прибыль','Повторение товара'],
                    index=["Факты.Товар ID"], aggfunc={'Продажи, шт': [np.median, np.sum], 'Продажи, руб': np.sum, 
                                                       'Повторение заказа': np.sum, 'Маржинальная прибыль': np.sum})

In [93]:
df.columns = df.columns.map('_'.join)
df = df.reset_index()

In [94]:
total_sale = df['Продажи, руб_sum'].sum()
df['Доля'] = df['Продажи, руб_sum'] / total_sale * 100
df = df.assign(sum_d=df['Доля'].cumsum())

In [95]:
df.loc[(df['sum_d'] <= 80), 'ABC'] = 'A'
df.loc[(df['sum_d'] > 80) & (df['sum_d'] <= 95), 'ABC'] = 'B'
df.loc[(df['sum_d'] > 95), 'ABC'] = 'C'

In [97]:
input_data = df.drop(['Факты.Товар ID','ABC'], axis=1)
output_data = df['ABC']

In [99]:
X_train, X_test, Y_train, Y_test = train_test_split(df_in, df_out, test_size=0.3)

In [101]:
# первый метод
model_1 = RandomForestClassifier()
model_1.fit(X_train, Y_train)
predict = model_1.predict(X_test)
print("Accuracy of RandomForestClassifier:", metrics.accuracy_score(Y_test, predict))

Accuracy of RandomForestClassifier: 1.0


In [103]:
# второй метод
model_2 = KNeighborsClassifier()
model_2.fit(X_train, Y_train)
predict = model_2.predict(X_test)
print("Accuracy of KNeighborsClassifier:", metrics.accuracy_score(Y_test, predict))

Accuracy of KNeighborsClassifier: 0.7280858676207513


In [106]:
# третий метод
model_3 = svm.SVC()
model_3.fit(X_train, Y_train)
predict = model_3.predict(X_test)
print("Accuracy of SupportVectorMachine:", metrics.accuracy_score(Y_test, predict))

Accuracy of SupportVectorMachine: 0.7620751341681574
